# Emotion Prediction from RGB Data Using CNN

This Jupyter Notebook demonstrates how to predict emotions from RGB data using a Convolutional Neural Network (CNN) in Keras. We will use this demmo data generated to simulate RGB values associated with different emotional states.

## Table of Contents
1. Importing Libraries
2. Loading and Preparing the Data
3. Data Preprocessing
4. Building the CNN Model
5. Training the Model
6. Evaluating the Model
7. Conclusion and Further Steps

## 1. Importing Libraries

In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import kerastuner as kt
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Loading and Preparing the Data

In [39]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Load the dataset
data = pd.read_csv('demmo_data.csv')

# Convert RGB strings to numerical arrays
def parse_rgb(rgb_str):
    return np.array(eval(rgb_str))

data['RGB_1'] = data['RGB_1'].apply(parse_rgb)
data['RGB_2'] = data['RGB_2'].apply(parse_rgb)
data['RGB_3'] = data['RGB_3'].apply(parse_rgb)

# Normalize the RGB values to [0, 1]
data['RGB_1'] = data['RGB_1'].apply(lambda x: x / 255.0)
data['RGB_2'] = data['RGB_2'].apply(lambda x: x / 255.0)
data['RGB_3'] = data['RGB_3'].apply(lambda x: x / 255.0)

# Combine RGB values into a single array
X = np.stack(data[['RGB_1', 'RGB_2', 'RGB_3']].values)
X = np.stack([np.stack(x, axis=0) for x in X])

# Ensure the data type is float32 and correct shape
X = X.astype('float32')
X = X.reshape(-1, 3, 3, 3)

# Encode the target labels
le = LabelEncoder()
y = data['Emotional_Word']
y_encoded = le.fit_transform(y)

# Convert to categorical (one-hot encoding)
y_categorical = to_categorical(y_encoded)

# Check the shape of the prepared data
print(f'X shape: {X.shape}, y shape: {y_categorical.shape}')

X shape: (76, 3, 3, 3), y shape: (228, 155)


### Step 3: Define the CNN Model with Keras Tuner

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(3, 3, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(le.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))


c:\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


ValueError: Found input variables with inconsistent numbers of samples: [76, 228]

### Step 4: Training the Model

In [32]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))


Epoch 1/50


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 3, 3), dtype=float32). Expected shape (None, 3, 3, 3), but input has incompatible shape (None, 3, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 3, 3), dtype=float32)
  • training=True
  • mask=None

### Step 4 :Model Evaluation and Prediction

In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {val_accuracy}')

# Function to predict emotional word for new RGB values
def predict_emotion(rgb1, rgb2, rgb3):
    rgb1, rgb2, rgb3 = np.array(rgb1) / 255.0, np.array(rgb2) / 255.0, np.array(rgb3) / 255.0
    new_data = np.stack([rgb1, rgb2, rgb3])[np.newaxis, :]
    prediction = model.predict(new_data)
    predicted_label = le.inverse_transform([np.argmax(prediction)])
    return predicted_label[0]

# Example prediction
new_rgb1 = [251, 167, 157]
new_rgb2 = [255, 242, 124]
new_rgb3 = [179, 22, 61]
print(f'Predicted Emotion: {predict_emotion(new_rgb1, new_rgb2, new_rgb3)}')


### Step 4: Initialize Keras Tuner and Search for Best Hyperparameters

In [4]:
# Train the simple CNN model
history = simple_cnn_model.fit(
    X_train_cnn,
    y_train,
    epochs=50,
    validation_data=(X_test_cnn, y_test),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
)

# Evaluate the model
loss, accuracy = simple_cnn_model.evaluate(X_test_cnn, y_test)
print(f'Test Accuracy: {accuracy}')


Epoch 1/50


ValueError: Exception encountered when calling Conv2D.call().

[1mNegative dimension size caused by subtracting 2 from 1 for '{{node sequential_1/conv2d_1_2/convolution}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](sequential_1/dropout_1/stateless_dropout/SelectV2, sequential_1/conv2d_1_2/convolution/ReadVariableOp)' with input shapes: [?,1,1,32], [2,2,32,64].[0m

Arguments received by Conv2D.call():
  • inputs=tf.Tensor(shape=(None, 1, 1, 32), dtype=float32)

### Step: Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def build_model(optimizer='adam', neurons=128):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(3, 3, 3)),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(neurons, activation='relu'),
        Dense(len(le.classes_), activation='softmax')
    ])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=build_model, epochs=50, batch_size=10, verbose=0)

param_dist = {
    'optimizer': ['adam', 'rmsprop'],
    'neurons': [64, 128, 256]
}

random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=5, cv=3)
random_search.fit(X_train, y_train)

print(f'Best Parameters: {random_search.best_params_}')


## 4. Building the CNN Model
##### I use a VGG-16-like architecture adapted for my dataset

## 5. Training the Model


## 6. Evaluating the Model

## 7. Conclusion and Further Steps
